<a href="https://colab.research.google.com/github/alfredashraf/AI_Code_Review/blob/main/AI_Code_Reviewer_Using_Ollama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Ollama**

In [ ]:
!nvidia-smi

Thu Aug  7 19:18:55 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   50C    P8             10W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!apt-get update
!apt-get install -y curl lspci

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [32.9 kB]
Get:12 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy/main amd64 Packages [35.7 kB]
Fetched 499 kB in 2s (239 kB/s

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [ ]:
!pip install ollama

In [ ]:
import subprocess
subprocess.Popen("ollama serve",shell=True)

<Popen: returncode: None args: 'ollama serve'>

In [ ]:
!ollama run gemma3:12b

hi

>>> Send a message (/? for help)hi
Hi there! 😊 

How can I help you today?

>>> Send a message (/? for help)exit
... 
Okay, exiting! Have a great day! 👋

>>> Send a message (/? for help)^C


In [ ]:
!pip install langchain-community pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.2/313.2 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00


In [ ]:
from langchain_community.llms import Ollama
from langchain_core.prompts import ChatPromptTemplate
from langchain.memory import ConversationBufferMemory
import subprocess
import time

In [ ]:
# === Step 1: Start the Ollama server ===
subprocess.Popen("ollama serve", shell=True)
time.sleep(15)  # Wait for Ollama to load the model

# === Step 2: Initialize the LLM ===
llm = Ollama(model="gemma3:12b")



/tmp/ipython-input-2590512466.py:6: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma3:12b")


In [ ]:
memory = ConversationBufferMemory(memory_key="history", return_messages=True)

# === Step 5: Create the prompt template ===
question_prompt_template =f"""
        You are an expert Python code reviewer. Analyze this code:
        ```python
        code_snippet
        ```
        If errors exist, identify them, explain the issues, and provide corrected code.
        If no errors, suggest improvements for structure, readability, or performance.
        Return in this format:
        - **Status**: [Errors Found / No Errors]
        - **Analysis**: [Explanation of errors or improvements]
        - **Code**: ```python
        [Corrected or improved code]
        ```
        """

evaluation_prompt_template='''You are a senior code reviewer.

Given the original code snippet, a reference reviewed version, and a submitted reviewed version,
evaluate how close the submitted version is to the reference version in terms of:

- Correctness
- Coverage of issues
- Quality of improvements
- Clarity of explanation

Please assign a score from 0 to 10 and provide reasoning.

### Original Code:
```python
def add(a, b):
return a + b
```

### Reference Review Response:
- **Status**: Errors Found
- **Analysis**: The return line is not properly indented. This will raise an IndentationError.
- **Code**:
```python
def add(a, b):
    return a + b
```

### Submitted Review Response:
- **Status**: No Errors
- **Analysis**: Code is fine.
- **Code**:
```python
def add(a, b):
return a + b
```

### Your Evaluation:
- **Score**: [0–10]
- **Rationale**: [Why this score was given]
'''

question_prompt = ChatPromptTemplate.from_template(question_prompt_template)
evaluation_prompt = ChatPromptTemplate.from_template(evaluation_prompt_template)


In [ ]:
# build chain
question_chain = (
    question_prompt
    | llm
)
evaluation_chain =(
     evaluation_prompt
     | llm
)

In [ ]:
# Generate formatted prompt
question = '''def add_numbers(x, y):

    resul  x + y
    return result'''

# Call the LLM (returns a plain string)
response = llm.invoke(question)


In [ ]:
print(response)

```python
def add_numbers(x, y):
   
    result = x + y
    return result
```

**Explanation:**

* **`def add_numbers(x, y):`**: This line defines a function named `add_numbers` that takes two arguments, `x` and `y`.
* **`result = x + y`**: This line calculates the sum of `x` and `y` and stores it in a variable named `result`.
* **`return result`**: This line returns the value of the `result` variable, which is the sum of `x` and `y`.

**How it works:**

When you call the `add_numbers` function with two numbers as arguments, the function calculates their sum and returns that sum as the result.

**Example:**

```python
sum_result = add_numbers(5, 3)
print(sum_result)  # Output: 8
```
**Note:**  The original code had a typo (`resul` instead of `result`).  I'm providing the corrected version.



In [ ]:
# Get user input
# user_reply = input("Paste your Python code for review:\n")

# Escape curly braces for safe formatting
# clean_code_snippet = user_reply.replace("{", "{{").replace("}", "}}")

# Load memory history (if needed)
current_history = memory.load_memory_variables({})["history"]

# Format prompt input
input_data = {
    # "code_snippet": clean_code_snippet,
    "history": current_history
}




# ✅ Print the actual response string
print("\n========= CODE REVIEW =========")
print(response)

# Save both Q and A to memory
memory.save_context(
    {"question": question},
    {"answer": response}
)



========= CODE REVIEW =========
```python
def add_numbers(x, y):
   
    result = x + y
    return result
```

**Explanation:**

* **`def add_numbers(x, y):`**: This line defines a function named `add_numbers` that takes two arguments, `x` and `y`.
* **`result = x + y`**: This line calculates the sum of `x` and `y` and stores it in a variable named `result`.
* **`return result`**: This line returns the value of the `result` variable, which is the sum of `x` and `y`.

**How it works:**

When you call the `add_numbers` function with two numbers as arguments, the function calculates their sum and returns that sum as the result.

**Example:**

```python
sum_result = add_numbers(5, 3)
print(sum_result)  # Output: 8
```
**Note:**  The original code had a typo (`resul` instead of `result`).  I'm providing the corrected version.



In [ ]:
# Define the code snippet you want to review
code_snippet = """
def add_numbers(x, y):

    result = x + y
    return result

# Example usage:
num1 = 5
num2 = 10
sum_result = add_numbers(num1, num2)
print(f"The sum of {num1} and {num2} is: {sum_result}")
"""